In [1]:
# 0. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Daftar Path Dataset
dataset_paths = [
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEA1_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEA2_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEA3_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEB1_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEB2_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEB3_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEC1_dataset_latent.csv',
    '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEC2_dataset_latent.csv',
    # '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEC3_dataset_latent.csv'
]

# 4. Fungsi Pelatihan dan Evaluasi
def train_and_evaluate(dataset_path):
    # Load Data
    data = pd.read_csv(dataset_path)
    X = data.drop('Diabetes_012', axis=1)
    y = data['Diabetes_012']
    y_encoded = to_categorical(y)

    # Split Data
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=SEED, stratify=y)
    X_train_main, X_val, y_train_main, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train.argmax(axis=1))

    # Arsitektur Model
    input_layer = Input(shape=(X.shape[1],))
    x = Dense(64, activation='relu')(input_layer)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output_layer = Dense(3, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train Model
    history = model.fit(
        X_train_main, y_train_main,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=50,
        callbacks=[early_stop],
        verbose=0
    )
    # 12. Simpan Model dalam Format .keras
    save_dir = '/content/drive/MyDrive/Projects/DNN untuk Prediksi status diabetes/Model'
    os.makedirs(save_dir, exist_ok=True)

    model_path = os.path.join(save_dir, 'dnn+kccae-c2.keras')
    model.save(model_path)

    # Last Epoch sebelum Early Stopping
    last_epoch = len(history.history['loss'])

    # Evaluasi
    y_pred_prob = model.predict(X_test)
    y_pred_class = np.argmax(y_pred_prob, axis=1)
    y_true_class = np.argmax(y_test, axis=1)

    report = classification_report(y_true_class, y_pred_class, digits=4, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_pred_prob, average='macro', multi_class='ovr')

    return report, roc_auc, last_epoch

# 5. Jalankan untuk Semua Dataset
for i, path in enumerate(dataset_paths, start=1):
    print(f"\n=== Evaluasi Dataset {i} ===")
    report, roc_auc, last_epoch = train_and_evaluate(path)
    print(f"Last Epoch (before early stopping): {last_epoch}")
    print(f"Accuracy: {report['accuracy']:.4f}")
    print(f"Macro Precision: {report['macro avg']['precision']:.4f}")
    print(f"Macro Recall: {report['macro avg']['recall']:.4f}")
    print(f"Macro F1-score: {report['macro avg']['f1-score']:.4f}")
    print(f"Macro-average ROC AUC: {roc_auc:.4f}")


Mounted at /content/drive

=== Evaluasi Dataset 1 ===
1586/1586 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Last Epoch (before early stopping): 26
Accuracy: 0.8475
Macro Precision: 0.4729
Macro Recall: 0.3718
Macro F1-score: 0.3763
Macro-average ROC AUC: 0.7574


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
